### 0. 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
df = sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


### 1. 데이터 전처리

- Feature Selection

In [4]:
df = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'deck']]
print(df.shape)
df.head(3)

(891, 9)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN


- 결측치 처리

In [5]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [6]:
# age 결측치를 평균값으로 치환
df['age'].fillna(df['age'].mean(), inplace=True)
df['age'].isnull().sum()

0

In [7]:
# embarked 결측치를 최빈값으로 치환
print(df['embarked'].value_counts())
df['embarked'].fillna('S', inplace=True)
df['embarked'].isnull().sum()

S    644
C    168
Q     77
Name: embarked, dtype: int64


0

In [8]:
# deck 열 삭제
df.drop(columns = ['deck'], inplace=True)

In [9]:
df.isna().sum().sum()

0

- 카테고리 값(sex, embarked)을 숫자로 변경

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
df.sex = le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


### 2. Train/Test dataset으로 분리

In [12]:
# X와 y를 넘파이 배열로
X = df.iloc[:, 1:].values
y = df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [17]:
# y값의 분포
np.unique(y, return_counts=True)

(array([0, 1]), array([549, 342]))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=2022)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [19]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([439, 273]))

### 3. RandomForest 모델로 학습

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=2022)
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2022,
 'verbose': 0,
 'warm_start': False}

In [21]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=2022)

### 4. 모델 예측 및 평가

In [22]:
rf.score(X_test, y_test)

0.8324022346368715

### 5. GridSearchCV 수행

In [23]:
params = {'max_depth': [2, 4, 6, 8], 'min_samples_split': [2, 4, 6]}

In [24]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rf, param_grid = params, scoring = 'accuracy', cv = 5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [25]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 4}

In [26]:
params = {'max_depth': [3, 4, 5], 'min_samples_split': [3, 4, 5]}
grid_rf = GridSearchCV(rf, param_grid = params, scoring = 'accuracy', cv = 5)
%time grid_rf.fit(X_train, y_train)
grid_rf.best_params_

CPU times: user 6.9 s, sys: 44.7 ms, total: 6.95 s
Wall time: 6.95 s


{'max_depth': 4, 'min_samples_split': 3}

In [27]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8212290502793296

### 6. 테스트 데이터에 적용

In [28]:
X_test[24], y_test[24]

(array([ 3.   ,  0.   , 40.   ,  1.   ,  0.   ,  9.475,  2.   ]), 0)

In [29]:
best_rf.predict(X_test[24].reshape(1, -1))[0]

0

### 7. 엉터리 분류기 만들기
- 여성이면 1, 남성이면 0으로 분류할 때의 정확도는 0.78임
- 모델의 성능을 정확도로만 평가하는 것은 지양해야함

In [33]:
# 여성의 생존율 확인
df.groupby('sex')['survived'].mean()

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

In [34]:
# 여성이면 무조건 생존하는 것으로 분류기 생성

from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    def fit(self, X, y):                # 학습 안 함
        pass 
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X[i, 1] == 0:            # 여성이면
                pred[i] = 1             # 생존
        return pred

In [35]:
my_clf = MyClassifier()
my_clf.fit(X_train, y_train)
pred_my = my_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

### 8. 평가

In [48]:
# RF, 엉터리 분류기 비교
pred_rf = best_rf.predict(X_test)
sdf = pd.DataFrame({'y_test': y_test, 'RF': pred_rf,'My': pred_my})
sdf.head()

,y_test,RF,My
0,1,0,0.0
1,0,0,0.0
2,1,0,1.0
3,0,0,0.0
4,0,0,0.0


In [ ]:
# 오차행렬
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_rf)

array([[103,   7],
       [ 25,  44]])

In [ ]:
confusion_matrix(y_test, pred_my)

array([[96, 14],
       [24, 45]])

In [49]:
# 정밀도(Precision score = TP / (FP + TP))
from sklearn.metrics import precision_score
precision_score(y_test, pred_rf), precision_score(y_test, pred_my)

(0.8627450980392157, 0.7627118644067796)

In [50]:
# 재현율(Recall score = TP / (FN + TP))
from sklearn.metrics import recall_score
recall_score(y_test, pred_rf), recall_score(y_test, pred_my)

(0.6376811594202898, 0.6521739130434783)

In [51]:
# F1 Score(정밀도와 재현율의 조화 평균)
from sklearn.metrics import f1_score
f1_score(y_test, pred_rf), f1_score(y_test, pred_my)

(0.7333333333333333, 0.703125)

In [52]:
# AUC Score(ROC 곡선의 밑 면적)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred_rf), roc_auc_score(y_test, pred_my)

(0.787022397891963, 0.7624505928853755)